In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 63.2 MB/s eta 0:00:00


In [4]:
import os
import rasterio
import math

path = '/content/drive/MyDrive/OmanChromiteResearch/WorldView/Pansharp_GROOVEY_truecolor_03m_052021.tif'

with rasterio.open(path) as dataset:
    # Calcular as dimensões de cada quadrante considerando a sobreposição
    # Primeiro, calculamos a largura e a altura de cada quadrante sem a sobreposição
    base_width_quadrant = dataset.width // 2
    base_height_quadrant = dataset.height // 2

    # Então adicionamos 10% do tamanho do quadrante como sobreposição
    width_quadrant = math.ceil(base_width_quadrant * 1.1)
    height_quadrant = math.ceil(base_height_quadrant * 1.1)

    # Iterando sobre cada quadrante na matriz de 2x2
    for i in range(2):
        for j in range(2):
            # Definir o nome do novo arquivo
            new_file_path = os.path.splitext(path)[0] + f'_quadrant_{i + 1}_{j + 1}.tif'

            # Calcular as coordenadas de origem do quadrante, considerando a sobreposição
            x_offset = max(0, i * base_width_quadrant - math.ceil(base_width_quadrant * 0.1))
            y_offset = max(0, j * base_height_quadrant - math.ceil(base_height_quadrant * 0.1))

            # Calcular a largura e altura do quadrante, considerando a sobreposição e as bordas da imagem
            quadrant_width = min(width_quadrant, dataset.width - x_offset)
            quadrant_height = min(height_quadrant, dataset.height - y_offset)

            # Atualizar a transformação para o novo quadrante
            new_transform = rasterio.Affine.translation(x_offset * dataset.transform.a, y_offset * dataset.transform.e) * dataset.transform

            # Definir os metadados para o novo arquivo
            new_metadata = dataset.meta.copy()
            new_metadata.update({
                'width': quadrant_width,
                'height': quadrant_height,
                'transform': new_transform
            })

            # Criar e escrever o novo arquivo GeoTIFF
            with rasterio.open(new_file_path, 'w', **new_metadata) as dest:
                for band_id in range(1, dataset.count + 1):
                    data = dataset.read(band_id, window=((y_offset, y_offset + quadrant_height), (x_offset, x_offset + quadrant_width)))
                    dest.write(data, band_id)
